### Model configuration:

- Detection scale-up function is multiplied by base diagnostic capacity
- GeneXpert utilization improvement is implemented
- COVID-related detection reduction is incorporated into the model
- For proportion of reported cases, two time points (2017 and 2023) are used with calibrated values for each time point

In [ ]:
import pandas as pd
from estival.wrappers.nevergrad import optimize_model
from tb_incubator.plotting import plot_model_vs_actual, display_plot, plot_tracked_outputs
from tb_incubator.input import load_targets, load_param_info
from tb_incubator.calibrate import get_bcm
from tb_incubator.utils import get_param_table, dict_to_markdown_table
from multiprocessing import cpu_count
pd.options.plotting.backend = "plotly"

In [ ]:
fixed_params = load_param_info()["value"]
param_info = load_param_info()
all_targets = load_targets()
covid_effects = {
    'detection_reduction':True
}
bcm = get_bcm(fixed_params, xpert_improvement=True, covid_effects=covid_effects)

In [ ]:
def calibrate_model_with_optimisation(bcm):
    """
    This function performs a model calibration using optimisation. 
    Calibration is performed using the estival package, which implements a wrapper for optimisation methods provided by the nevergrad package. 

    Args:
        bcm: the calibration model object (type BayesianCompartmentalModel) 
    """

    from nevergrad.optimization.differentialevolution import TwoPointsDE
    orunner = optimize_model(bcm, opt_class=TwoPointsDE, num_workers=cpu_count(), budget=4000)
    rec = orunner.minimize(4000)
    optimised_params = rec.value[1]    
   
    return optimised_params

In [ ]:
optimised_params = calibrate_model_with_optimisation(bcm)
#optimised_params

In [ ]:
from IPython.display import Markdown, display

param_table = dict_to_markdown_table(optimised_params)

display(Markdown(param_table))

In [ ]:
# run the modle with the optimised parameter set
res = bcm.run(optimised_params)
outs = res.derived_outputs

### Results

In [ ]:
fig = plot_model_vs_actual(
    outs, all_targets['notification'], "notification", "Notification", "Modelled vs Data", "Target data"
)
fig.update_xaxes(range=[1970, 2035])
#display_plot(fig, "genexpert_2points_notif")

In [ ]:
fig = plot_model_vs_actual(
    outs, all_targets["adults_prevalence_pulmonary_target"], "adults_prevalence_pulmonary", "Prevalence (Adult, Pulmonary)", "Modelled vs Data", "Target data"
)
fig.update_xaxes(range=[1970, 2035])

In [ ]:
tracked_outputs = [
    "base_detection",
    "diagnostic_improvement",
    "diagnostic_capacity",
    "covid_impacts",
    "final_detection",
    "notif_ratio"
]

plot_tracked_outputs(outs, tracked_outputs, layout=(3,2), plot_start_date=2000)